In [4]:
import numpy as np
import math
from sklearn import datasets
from collections import Counter
from sklearn.model_selection import train_test_split
infinity = float(-2 ** 31)

## python实现POLY2

In [37]:
def sigmoid_train(X, b, W, W_join):
    #训练阶段 用当前的权重预测样本X的每个样本的概率
    #一阶部分
    sum_linear = b + X.dot(W)
    samples = len(X)
    features = len(X[0])
    #二阶特征组合
    X_join = []
    for sample_index in range(samples):
        sample = X[sample_index]
        sample_join = []
        for fea_j1_index in range(features):
            for fea_j2_index in range(fea_j1_index + 1, features):
                sample_join.append(sample[fea_j1_index] * sample[fea_j2_index])
        X_join.append(sample_join)
    X_join = np.array(X_join)
    #二阶部分z值
    sum_join = X_join.dot(W_join)
    params = -(sum_linear + sum_join)
    r = np.zeros(params.shape[0]) #生成维度为样本数的一维数组
    for i in range(len(r)):
        r[i] = 1 / (1 + math.exp(params[i]))
    return r
def sigmoid_predict(X, b, W, W_join):
    #预测阶段 用当前W预测X的每个样本的标签，概率值大于等于0.5标签为1 否则标签为0
    sum_linear = b + X.dot(W)
    samples = len(X)
    features = len(X[0])
    #二阶特征组合
    X_join = []
    for sample_index in range(samples):
        sample = X[sample_index]
        sample_join = []
        for fea_j1_index in range(features):
            for fea_j2_index in range(fea_j1_index + 1, features):
                sample_join.append(sample[fea_j1_index] * sample[fea_j2_index])
        X_join.append(sample_join)
    X_join = np.array(X_join)
    #二阶部分z值
    sum_join = X_join.dot(W_join)
    params = -(sum_linear + sum_join)
    r = np.zeros(params.shape[0])
    for i in range(len(r)):
        r[i] = 1 / (1 + math.exp(params[i]))
        if r[i] >= 0.5:
            r[i] = 1
        else:
            r[i] = 0
    return r
def sigmoid(Xi, b, W, W_join):
    #计算某个样本在权重W下的概率
    #一阶部分值
    sum_linear = b + np.sum(Xi * W)
    #二阶特征组合
    features = len(Xi)
    X_join = []
    for fea_j1_index in range(features):
        for fea_j2_index in range(fea_j1_index + 1, features):
            X_join.append(Xi[fea_j1_index] * Xi[fea_j2_index])
    X_join = np.array(X_join)
    #二阶部分值
    sum_join = np.sum(X_join * W_join)
    params = -(sum_linear + sum_join)
    r = 1 / (1 + math.exp(params))
    return r

class POLY2(object):
    W = None
    b = 0
    W_join = None
    m = 0
    #训练
    #alpha：学习率,accuracy时训练停止条件
    def fit(self, X, y, alpha=0.01, accuracy=0.0001):
        #插入第一列为1(即偏置项b)，构成Xb矩阵
        self.W = np.full(X.shape[1], 0.5)#用0.5初始化一阶特征的权重向量W
        self.m = X.shape[0] #样本数量
        self.b = 1 #生成一列偏置项
        dimension = X.shape[1] #输入数据的特征个数
        self.W_join = np.full(dimension * (dimension - 1) // 2, 0.2)#用0.2初始化二阶组合特征的权重向量W_join
        #梯度下降迭代
        count = 1
        while True:
            oldJ = self.costFunc(X, y)
            #根据上面的公式更新偏置项，一阶权重，二阶组合权重
            error = sigmoid_train(X, self.b, self.W, self.W_join) - y
            self.b -= alpha * np.sum(error) / self.m
            for j in range(dimension):
                self.W[j] -= alpha * np.sum(error * X[:,j]) / self.m
            k = 0
            for j1 in range(dimension):
                for j2 in range(j1 + 1, dimension):
                    self.W_join[k] -= alpha * np.sum(error * X[:,j1] * X[:,j2]) / self.m
                    k += 1
            newJ = self.costFunc(X, y)
            if newJ == oldJ or math.fabs(newJ - oldJ) < accuracy:
                print('代价函数迭代到最小值，退出!')
                print('收敛到:', newJ)
                break
            print('迭代第', count, '次')
            print('代价函数上一次的差:', (oldJ - newJ))
            count += 1
            
    ##计算损失函数
    def costFunc(self, X, y):
        sums = 0.0
        for i in range(self.m):
            yPre = sigmoid(X[i,:], self.b, self.W, self.W_join)
            if yPre == 1 or yPre == 0:
                return infinity
            sums += y[i] * math.log(yPre) + (1 - y[i]) * math.log(1 - yPre)
        return -1 * sums / self.m
    #预测
    def predict(self, X):
        return sigmoid_predict(X, self.b, self.W, self.W_join)
    #用准确率衡量预测结果的好坏
    def score(self, X_test, y_test):
        y_predict = self.predict(X_test)
        re = (y_test==y_predict)
        re1 = Counter(re)
        acc = re1[True] / (re1[True] + re1[False])
        return acc

In [38]:
iris = datasets.load_iris()
X = iris['data']
y = iris['target']
X = X[y!=2]
y = y[y!=2]
X_train, X_test, y_train, y_test = train_test_split(X, y)

myPoly = POLY2()
myPoly.fit(X_train, y_train)
y_predict = myLogistic.predict(X_test)

print('偏置项:', myPoly.b)
print('一阶权重值:', myPoly.W)
print('二阶组合特征权重值:', myPoly.W_join)
print('测试集准确度:', myPoly.score(X_test, y_test))

迭代第 1 次
代价函数上一次的差: 0.8094566333466062
迭代第 2 次
代价函数上一次的差: 0.8093638538877777
迭代第 3 次
代价函数上一次的差: 0.8088782984930427
迭代第 4 次
代价函数上一次的差: 0.8061375254411893
迭代第 5 次
代价函数上一次的差: 0.7896386684761383
迭代第 6 次
代价函数上一次的差: 0.6976315245236709
迭代第 7 次
代价函数上一次的差: 0.41553711924151787
迭代第 8 次
代价函数上一次的差: 0.13915077280168106
迭代第 9 次
代价函数上一次的差: 0.04052315016222954
迭代第 10 次
代价函数上一次的差: 0.014206442270451314
迭代第 11 次
代价函数上一次的差: 0.00831075886721977
迭代第 12 次
代价函数上一次的差: 0.006851980883903541
迭代第 13 次
代价函数上一次的差: 0.006129395371694296
迭代第 14 次
代价函数上一次的差: 0.005569343532794549
迭代第 15 次
代价函数上一次的差: 0.005088931204833114
迭代第 16 次
代价函数上一次的差: 0.004668377995375561
迭代第 17 次
代价函数上一次的差: 0.004297473433751031
迭代第 18 次
代价函数上一次的差: 0.003968669213687631
迭代第 19 次
代价函数上一次的差: 0.003675876666944855
迭代第 20 次
代价函数上一次的差: 0.0034140739188478486
迭代第 21 次
代价函数上一次的差: 0.003179079314096417
迭代第 22 次
代价函数上一次的差: 0.002967387458556278
迭代第 23 次
代价函数上一次的差: 0.0027760427623082923
迭代第 24 次
代价函数上一次的差: 0.0026025398580026005
迭代第 25 次
代价函数上一次的差: 0.0024447443618396